In [ ]:
#for phoebe to recognize her name if spoken ('always on' mode)
!pip install pydub

from pydub import AudioSegment

from pydub.silence import split_on_silence
import os

!pip install SpeechRecognition
!pip install pipwin
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
#!pip install PyAudio
!pipwin install pyaudio

import speech_recognition as s
from scipy.io import wavfile
import matplotlib.pyplot as plt


In [ ]:
#concatenate background sounds in wav format

fnames =["cafe.wav", "newspaper.wav", "Indoor-Restaurant.wav"]
sound = sum([AudioSegment.from_wav(f) for f in fnames])

sound.export('mybackground.wav', format="wav")



In [ ]:
#overlay audio with background

sound1 = AudioSegment.from_file("Voice-003.wav")
sound2 = AudioSegment.from_file("mybackground.wav")

combined = sound1.overlay(sound2)

combined.export("training.wav", format='wav')


In [ ]:
#split the stream into different inputs (rows)

def split(filepath):
    sound = AudioSegment.from_wav(filepath)
    dBFS = sound.dBFS
    chunks = split_on_silence(sound, 
        min_silence_len = 80,
        silence_thresh = dBFS-16,
        keep_silence = 250)
    return chunks
    
    
filepath = "training.wav"
chunks = split(filepath)

In [ ]:
#save the split files
for i, chunk in enumerate(chunks):
    chunk.export(os.path.join("phoebe training chunks/", "audio" + "{0}.wav".format("%02d" % i)), format="wav")

In [ ]:
#see how the audios are recognised - speech-to-text

r = s.Recognizer()
l = []
for f in os.listdir('phoebe training chunks/'):
    with s.AudioFile('phoebe training chunks/'+f) as source:
        audio = r.record(source)
        try:
            l.extend([r.recognize_google(audio)])
        except:
            l.extend([0]) #take 0 if can't recognise

In [ ]:
print(l)

In [ ]:

#create images from .wav file (already defined audios)

for f in os.listdir('phoebe training chunks/') :
  
  samplingFrequency, signalData = wavfile.read('phoebe training chunks/'+f)

  plt.title('Spectrogram')

  plt.specgram(signalData[:,0],Fs=samplingFrequency,NFFT=512)

  plt.xlabel('Time')

  plt.ylabel('Frequency')

  plt.savefig('phoebe_training_spec/'+ f.split('.')[0]+'.png') #save files into one folder